In [ ]:
!pip install split_folders

import os
import zipfile
import splitfolders
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.callbacks import Callback

import warnings
warnings.filterwarnings('ignore')

In [ ]:
!wget "https://github.com/dicodingacademy/assets/releases/download/release/rockpaperscissors.zip"

In [ ]:
local_zip = "/content/rockpaperscissors.zip"
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall()
zip_ref.close()

In [ ]:
main_directory = "/content/rockpaperscissors"

print(os.listdir(main_directory))

In [ ]:
splitfolders.ratio('/content/rockpaperscissors/rps-cv-images', 
                   output="/content/data", 
                   seed=1337, 
                   ratio=(.6, .4)
                   )

In [ ]:
train_dir = "/content/data/train"
valid_dir = "/content/data/val"

In [ ]:
train_image = 0

for dir in os.listdir(train_dir):
  train_image += len(os.listdir(os.path.join(train_dir, dir)))

print(train_image)

In [ ]:
valid_image = 0

for dir in os.listdir(valid_dir):
  valid_image += len(os.listdir(os.path.join(valid_dir, dir)))

print(valid_image)

In [ ]:
train_ds = image_dataset_from_directory(
    directory = "/content/data/train",
    batch_size = 32,
    label_mode = 'int'
)

In [ ]:
valid_ds = image_dataset_from_directory(
    directory = "/content/data/val",
    batch_size = 32,
    label_mode = 'int'
)

In [ ]:
print(train_ds.class_names)